In [27]:
!pip install facebook-sdk


In [28]:
import requests, facebook, json, time

# Getting User Token for Facebook Graph API
user_token = 'EAAKh8qpAhWIBAO0ebs5v95VjWoW9LOVhY1uIzhbl5Erfoj0mMHb7M22B3gxNEUX3LaMo6fKqslRRqMOOSPTxx37v4Qo83sV7egB7jBe3aWODZBZA7cNbNvZBqtnsnmC7EnuciYRxgLiPs4gyEa1duZAQrDhMuK529aUZArcthlyO2eYz7w3cg5yjluJPcz2yPSRzvKzT89BROjMb8cDenPGJZB9QIZBSlQWcZC7uRuB6ZCyy5cN7my5lGqtaogW2TvW0ZD'
graph = facebook.GraphAPI(user_token)
profile = graph.get_object("me", fields="posts")
posts_dict = profile['posts']

In [29]:
# This Function retrieves all the posts from one page
def get_page_posts(posts_dict):
    posts = posts_dict['data']
    post_infos = []

    for post in posts:
        try:
            created_date = post['created_time'][:10]
            message = post['message']
            url = "https://www.facebook.com/" + post['id']
        except:
            message = None


        if message:
            comb = [created_date, "Facebook", "Post", url, message]
            post_infos.append(comb)
        

    return post_infos

In [30]:
# This Function returns all the posts from all the pages
def get_all_posts(posts_dict):
    pages = posts_dict['paging']
    data = posts_dict['data']
    new_post_dict = posts_dict
    all_posts =[]


    while data != []:
        page_posts = get_page_posts(new_post_dict)
        all_posts.extend(page_posts)

        next_page = pages['previous']

        next_request = requests.get(next_page)

        new_post_dict = next_request.json()
        data = new_post_dict['data']

    return all_posts

In [31]:
posts = get_all_posts(posts_dict)

In [32]:
print(posts)

[['2022-04-10', 'Facebook', 'Post', 'https://www.facebook.com/107163565298907_107930128555584', 'cool story bro. In what chapter do you shut the fuck up?'], ['2022-04-10', 'Facebook', 'Post', 'https://www.facebook.com/107163565298907_107930088555588', 'turn your wounds into wisdom'], ['2022-04-10', 'Facebook', 'Post', 'https://www.facebook.com/107163565298907_107929985222265', 'our love was lost. Your heart turned cold'], ['2022-04-10', 'Facebook', 'Post', 'https://www.facebook.com/107163565298907_107929815222282', 'what a job.'], ['2022-04-10', 'Facebook', 'Post', 'https://www.facebook.com/107163565298907_107929778555619', "I love this Pizza, who's the chef?"], ['2022-04-10', 'Facebook', 'Post', 'https://www.facebook.com/107163565298907_107929668555630', 'yo Check THIS Out!! Hit me up!! 010-XXXX-XXXX'], ['2022-04-10', 'Facebook', 'Post', 'https://www.facebook.com/107163565298907_107929545222309', "have you heard of Elon Musk's Neural link? That is fucking awesome!!"], ['2022-04-10', '

### 


In [33]:
# installing requirements
!pip install nltk
!pip install better-profanity
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from better_profanity import profanity
import pandas as pd

# loading train data
review_df = pd.read_csv(r'C:\Users\seung\Code\yhack2022\Untitled Folder\review_data.txt', header = 0, sep = "\t", quoting = 3)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\seung\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
# porfanity and personal information filter
personal_info=['010-xxxx-xxxx']

# profanity_counter = 0
# personal_information_counter = 0

def checking(words_list):
    filter_list=[]
    profanity_counter = 0
    personal_information_counter = 0
    
    content = word_tokenize(words_list)

    for element in content:
        
        if profanity.contains_profanity(element) == True:
            profanity_counter += 1
        if element in personal_info:
            personal_information_counter += 1
    if profanity_counter > 0 :
        filter_list.append("profanity")

    if personal_information_counter > 0 :
        filter_list.append("personal information")
    
    return filter_list


In [36]:
filter_list2=[]
for k in range(len(posts)):
    filter_list2.append(checking(posts[k][-1]))



In [38]:
 #unstable mood filter
import re

# <br> html 태그는 replace 함수로 공백으로 변환
review_df['review'] = review_df['review'].str.replace('<br />', ' ')

# 파이썬의 re 모듈을 이용해 영어 문자열이 아닌 문자는 모두 공백으로 변환
review_df['review'] = review_df['review'].apply(lambda x : re.sub("[^a-zA-Z]", " ", x))


TypeError: expected string or bytes-like object

In [21]:
# 결정 값 클래스인 sentiment 칼럼을 별도로 추출
from sklearn.model_selection import train_test_split


# 학습 데이터에서 sentiment만 추출
class_df = review_df['sentiment']
# 학습 데이터에서 review만 추출
feature_df = review_df.drop(['id','sentiment'], axis=1, inplace=False)

X_train, X_test, y_train, y_test= train_test_split(feature_df, class_df, test_size=0.2, random_state=156,shuffle=True)
X_train.shape, X_test.shape


((20044, 1), (5011, 1))

In [22]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# 스톱 워드는 English, filtering, ngram은 (1,2)로 설정해 CountVectorization수행. 
# LogisticRegression의 C는 10으로 설정. 
pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(C=2, max_iter = 100))]) # C 값이 작으면 훈련을 덜 복잡 ( 강한 규제)

# Pipeline 객체를 이용하여 fit(), predict()로 학습/예측 수행. predict_proba()는 roc_auc때문에 수행.  
pipeline.fit(X_train['review'], y_train)
pred =pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}'.format(accuracy_score(y_test ,pred)))

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [47]:
filter_list2
posts[0][-1]

'cool story bro. In what chapter do you shut the fuck up?'

In [54]:
#pipeline.fit(X_train['review'], y_train)
for i, post in enumerate(posts):
    print(post[-1])
    pred =pipeline.predict([post[-1]])
    if pred == [0]:
        filter_list2[i].append("unstable mood")
    #해당 메소드를 실행하면서 학습한 데이터를 기반으로 주어진 Feature가 어떤 확률로 Label로 분류되었는지 계산하여 반환
    pipeline.predict_proba([post[-1]])

cool story bro. In what chapter do you shut the fuck up?
turn your wounds into wisdom
our love was lost. Your heart turned cold
what a job.
I love this Pizza, who's the chef?
yo Check THIS Out!! Hit me up!! 010-XXXX-XXXX
have you heard of Elon Musk's Neural link? That is fucking awesome!!
thanks for you help. I really appreciate it
I love dogs. They are very cute.
I'm broke. Would anyone raise a fund for me? Call me. 010-xxxx-xxxx
my life has no hope. I should just go die
I hate my mom's beef stew it's fucking disgusting
This account is a test account for app development.


In [55]:
# 어떤 확률로 선택했는지 찾는 코드 

def ref(x,y):
    return x if x>=y else y

for post in posts:
    percentage1=pipeline.predict_proba([post[-1]])[:,0]
    percentage2=pipeline.predict_proba([post[-1]])[:,1]
    
    print("test_sentance: ", post[-1],'\n',
     "sentiment :", pred, "\n",
      "percentage:", ref(percentage1,percentage2)*100,"[%]"
     )



test_sentance:  cool story bro. In what chapter do you shut the fuck up? 
 sentiment : [0] 
 percentage: [60.61100304] [%]
test_sentance:  turn your wounds into wisdom 
 sentiment : [0] 
 percentage: [54.63695746] [%]
test_sentance:  our love was lost. Your heart turned cold 
 sentiment : [0] 
 percentage: [66.33340485] [%]
test_sentance:  what a job. 
 sentiment : [0] 
 percentage: [69.13341775] [%]
test_sentance:  I love this Pizza, who's the chef? 
 sentiment : [0] 
 percentage: [68.38694565] [%]
test_sentance:  yo Check THIS Out!! Hit me up!! 010-XXXX-XXXX 
 sentiment : [0] 
 percentage: [58.78463033] [%]
test_sentance:  have you heard of Elon Musk's Neural link? That is fucking awesome!! 
 sentiment : [0] 
 percentage: [69.82025891] [%]
test_sentance:  thanks for you help. I really appreciate it 
 sentiment : [0] 
 percentage: [73.33966619] [%]
test_sentance:  I love dogs. They are very cute. 
 sentiment : [0] 
 percentage: [56.62183206] [%]
test_sentance:  I'm broke. Would anyone